# Authorization and Capabilities in Authority Nanos

This notebook explains the capability-based authorization system in Authority Nanos.

## What is Capability-Based Authorization?

Traditional security systems use **access control lists (ACLs)** - asking "who is allowed to do what?"

Authority Nanos uses **capabilities** - unforgeable tokens that grant specific permissions. Think of it like:

- **ACL**: "Show me your ID to prove you can enter"
- **Capability**: "Here's a key that opens this specific door"

### Benefits of Capabilities

1. **Principle of Least Privilege** - Grant only the permissions needed
2. **No Ambient Authority** - Can't accidentally use permissions you shouldn't
3. **Delegation** - Can pass capabilities to sub-agents
4. **Audit Trail** - Every capability use is logged

Let's explore how this works!

In [ ]:
# Install the SDK (uncomment if needed)
# %pip install authority-nanos

import json
from authority_nanos import (
    AuthorityKernel,
    OperationDeniedError,
    AuthorityKernelError
)

print("Authority Nanos SDK imported successfully!")

## 1. Basic Authorization Checks

Use `authorize(operation, target)` to check if an operation is allowed:

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    # In simulation mode, everything is allowed by default
    
    # Check file read authorization
    if ak.authorize("read", "/etc/config.json"):
        print("[ALLOWED] Can read /etc/config.json")
    else:
        print("[DENIED] Cannot read /etc/config.json")
    
    # Check HTTP authorization
    if ak.authorize("http.post", "https://api.example.com/data"):
        print("[ALLOWED] Can POST to https://api.example.com/data")
    else:
        print("[DENIED] Cannot POST to https://api.example.com/data")
    
    # Check write authorization
    if ak.authorize("write", "/tmp/output.txt"):
        print("[ALLOWED] Can write to /tmp/output.txt")
    else:
        print("[DENIED] Cannot write to /tmp/output.txt")

## 2. Simulating Policy Denials

In simulation mode, you can configure denials to test how your code handles them:

- `deny_operation(op)` - Deny all requests with this operation
- `deny_target(target)` - Deny all requests to this target
- `allow_all()` - Reset to allow everything

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    print("=== Denying Operations ===")
    print()
    
    # Initially, writes are allowed
    print("Before deny_operation:")
    print(f"  write /tmp/file.txt: {'ALLOWED' if ak.authorize('write', '/tmp/file.txt') else 'DENIED'}")
    print(f"  read /tmp/file.txt: {'ALLOWED' if ak.authorize('read', '/tmp/file.txt') else 'DENIED'}")
    
    # Deny all write operations
    ak.deny_operation("write")
    
    print()
    print("After deny_operation('write'):")
    print(f"  write /tmp/file.txt: {'ALLOWED' if ak.authorize('write', '/tmp/file.txt') else 'DENIED'}")
    print(f"  read /tmp/file.txt: {'ALLOWED' if ak.authorize('read', '/tmp/file.txt') else 'DENIED'}")
    
    # Reset
    ak.allow_all()
    print()
    print("After allow_all():")
    print(f"  write /tmp/file.txt: {'ALLOWED' if ak.authorize('write', '/tmp/file.txt') else 'DENIED'}")

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    print("=== Denying Specific Targets ===")
    print()
    
    # Deny access to sensitive files
    ak.deny_target("/etc/shadow")
    ak.deny_target("/etc/passwd")
    ak.deny_target("https://internal.corp.com")
    
    targets_to_check = [
        ("read", "/etc/shadow"),
        ("read", "/etc/passwd"),
        ("read", "/etc/hostname"),  # Not denied
        ("http.get", "https://internal.corp.com"),
        ("http.get", "https://api.public.com"),  # Not denied
    ]
    
    for op, target in targets_to_check:
        allowed = ak.authorize(op, target)
        status = "ALLOWED" if allowed else "DENIED"
        print(f"  {op} {target}: {status}")

## 3. Getting Denial Information

When an operation is denied, you can get details about why:

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    # Set up a denial
    ak.deny_target("/secrets/api_key.txt")
    
    # Try to access the denied resource
    if not ak.authorize("read", "/secrets/api_key.txt"):
        # Get denial information
        denial = ak.get_last_denial()
        
        if denial:
            print("Authorization Denied!")
            print(f"  Reason: {denial.reason}")
            print(f"  Operation: {denial.operation}")
            print(f"  Target: {denial.target}")

## 4. Handling Denial Exceptions

When you try to perform an operation that's denied, an `OperationDeniedError` is raised:

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    # Deny write operations
    ak.deny_operation("write")
    
    # Try to write a file (will be denied)
    try:
        ak.file_write("/tmp/test.txt", b"Hello World")
        print("File written successfully")
    except OperationDeniedError as e:
        print(f"Operation denied: {e}")
        print()
        print("Handling the denial gracefully...")
        print("Perhaps ask the user for permission or use a fallback.")

## 5. Pattern-Based Authorization (Advanced)

The `SimulatedKernel` from the `simulator` module supports regex-based authorization patterns:

In [ ]:
from authority_nanos.simulator import SimulatedKernel

with SimulatedKernel(
    deny_patterns=[
        r".*secret.*",       # Deny anything with "secret" in it
        r"write:/etc/.*",    # Deny writes to /etc/
        r"http.*:.*internal\.corp.*"  # Deny HTTP to internal.corp
    ]
) as ak:
    print("=== Pattern-Based Authorization ===")
    print()
    
    checks = [
        ("read", "/home/user/secrets.txt"),     # Matches 'secret'
        ("read", "/home/user/config.txt"),      # Allowed
        ("write", "/etc/passwd"),               # Matches 'write:/etc/'
        ("write", "/tmp/output.txt"),           # Allowed
        ("http.get", "https://internal.corp.com/api"),  # Matches internal.corp
        ("http.get", "https://api.public.com"),  # Allowed
    ]
    
    for op, target in checks:
        allowed = ak.authorize(op, target)
        status = "ALLOWED" if allowed else "DENIED"
        print(f"  {op:12} {target:40} {status}")

### Allow Patterns (Override Denials)

You can also add allow patterns that override denials:

In [ ]:
from authority_nanos.simulator import SimulatedKernel

with SimulatedKernel(
    deny_patterns=[
        r"write:.*",  # Deny all writes
    ],
    allow_patterns=[
        r"write:/tmp/.*",  # But allow writes to /tmp/
    ]
) as ak:
    print("=== Allow Patterns Override Denials ===")
    print()
    
    checks = [
        ("write", "/etc/passwd"),      # Denied by 'write:.*'
        ("write", "/var/log/app.log"), # Denied by 'write:.*'
        ("write", "/tmp/output.txt"),  # Allowed by 'write:/tmp/.*'
        ("write", "/tmp/cache.json"),  # Allowed by 'write:/tmp/.*'
    ]
    
    for op, target in checks:
        allowed = ak.authorize(op, target)
        status = "ALLOWED" if allowed else "DENIED"
        print(f"  {op:8} {target:25} {status}")

## 6. Audit Logging

All authorization checks are logged for auditing:

In [ ]:
with AuthorityKernel(simulate=True) as ak:
    # Perform some operations
    ak.deny_target("/secrets/key.txt")
    
    ak.authorize("read", "/public/data.json")
    ak.authorize("read", "/secrets/key.txt")
    ak.authorize("write", "/tmp/output.txt")
    
    # Get audit logs
    logs = ak.audit_logs()
    
    print("=== Audit Log ===")
    print()
    for log_bytes in logs[-5:]:  # Show last 5 entries
        entry = json.loads(log_bytes.decode())
        event = entry.get("event", "unknown")
        details = entry.get("details", {})
        
        if "authorized" in details:
            status = "ALLOWED" if details["authorized"] else "DENIED"
            op = details.get("operation", "?")
            target = details.get("target", "?")
            print(f"  [{event}] {op} {target}: {status}")
        else:
            print(f"  [{event}] {details}")

## 7. Practical Example: Secure File Operations

Here's a practical pattern for handling authorization:

In [ ]:
def read_file_safely(ak, path):
    """Read a file with proper authorization checking."""
    
    # Check authorization first
    if not ak.authorize("read", path):
        denial = ak.get_last_denial()
        print(f"Cannot read {path}")
        if denial:
            print(f"  Reason: {denial.reason}")
        return None
    
    # Authorized - proceed with read
    try:
        data = ak.file_read(path)
        return data
    except Exception as e:
        print(f"Error reading {path}: {e}")
        return None


# Test the function
with AuthorityKernel(simulate=True) as ak:
    # Allow /tmp but deny /secrets
    ak.deny_target("/secrets/credentials.json")
    
    print("=== Secure File Operations ===")
    print()
    
    # This will succeed (simulated file exists)
    print("Attempting to read /etc/config.json:")
    data = read_file_safely(ak, "/etc/config.json")
    if data:
        print(f"  Success! Read {len(data)} bytes")
    
    print()
    
    # This will be denied
    print("Attempting to read /secrets/credentials.json:")
    data = read_file_safely(ak, "/secrets/credentials.json")

## 8. Policy Configuration (Conceptual)

In production, policies are configured via TOML or JSON files. Here's what a policy might look like:

```toml
# policy.toml

[agent.my_agent]
description = "My AI Agent"

# Allow file operations in specific directories
[[agent.my_agent.allow]]
operation = "read"
target = "/data/*"

[[agent.my_agent.allow]]
operation = "write"
target = "/tmp/*"

# Allow specific HTTP endpoints
[[agent.my_agent.allow]]
operation = "http.get"
target = "https://api.weather.com/*"

# Deny everything else by default
[[agent.my_agent.deny]]
operation = "*"
target = "*"
```

See the Policy documentation for more details on configuring production policies.

## Summary

In this notebook, you learned:

1. **Capability-based authorization** - Permissions as unforgeable tokens
2. **Basic authorization checks** - Using `authorize(operation, target)`
3. **Simulating denials** - Using `deny_operation()` and `deny_target()`
4. **Denial information** - Using `get_last_denial()` for details
5. **Exception handling** - Catching `OperationDeniedError`
6. **Pattern-based policies** - Using regex patterns for fine-grained control
7. **Audit logging** - Tracking all authorization decisions

## Key Takeaways

- Always check authorization before performing sensitive operations
- Handle denials gracefully with informative error messages
- Use the audit log to track and debug authorization decisions
- In simulation mode, configure denials to test your error handling

## Next Steps

- **03_building_agents.ipynb** - Build AI agents with these authorization patterns
- **04_langchain_integration.ipynb** - Integrate with LangChain